In [1]:
%run imports.ipynb
%run '08222019_functions.ipynb'

In [2]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=30)
cluster

In [3]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://10.35.63.174:34937 Dashboard: /user/carlo%20mariade%20marco1/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
metadata = pd.read_csv("samples.meta.txt", sep="\t")
pos_all = allel.SortedIndex(callset_biallel['3R']["variants/POS"])
pos_all

0,1,2,3,4,...,10752696,10752697,10752698,10752699,10752700
9526,9531,9544,9546,9549,...,53195257,53195262,53195269,53195273,53195279


----------------------------------------
First way.

In [ ]:
loc = pos_all.locate_range(1_000_000, 24_000_000)
geno = allel.GenotypeChunkedArray(callset_biallel['3R']["calldata/GT"][loc]) 

### CRASH also using GenotypeDaskArray

-------------------------------------------------

Second way.

In [5]:
loc = pos_all.intersect_range(1_000_000, 24_000_000)
loc1, loc2 = pos_all.locate_intersection(loc)
geno = allel.GenotypeChunkedArray(callset_biallel['3R']["calldata/GT"])
geno_subset = geno.subset(sel0=loc1)
geno_subset

### It works!!!

<GenotypeChunkedArray shape=(5506386, 1142, 2) dtype=int8 chunks=(2689, 1142, 2)
   nbytes=11.7G cbytes=485.4M cratio=24.7
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

Dask:

In [ ]:
loc = pos_all.intersect_range(1_000_000, 24_000_000)
loc1, loc2 = pos_all.locate_intersection(loc)
geno = allel.GenotypeDaskArray(callset_biallel['3R']["calldata/GT"])
geno_subset = geno.subset(sel0=loc1)
geno_subset.compute()

### CRASH!!!!

-------------------------------------------------------
Third way:

In [ ]:
loc = pos_all.locate_range(1_000_000, 24_000_000)
geno = allel.GenotypeChunkedArray(callset_biallel['3R']['calldata/GT'])
subpops = metadata.groupby('population').indices
ac_subpops = geno[loc].count_alleles_subpops(subpops, max_allele=3)
ac_subpops

### CRASH!!!!

In [9]:
loc = pos_all.locate_range(1_000_000, 24_000_000)
gt = allel.GenotypeDaskArray(callset_biallel['3R']['calldata/GT'])
subpops = metadata.groupby('population').indices
ac_subpops = gt[loc].count_alleles_subpops(subpops, max_allele=3)
ac_subpops['AOcol'].compute()

<AlleleCountsArray shape=(5506386, 4) dtype=int32>
156   0   0   0
156   0   0   0
156   0   0   0
...
156   0   0   0
156   0   0   0
156   0   0   0

In [11]:
ac_subpops = gt[loc].count_alleles_subpops(subpops, max_allele=3)
ac_subpops

{'AOcol': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'BFcol': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'BFgam': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'CIcol': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'CMgam': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'FRgam': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'GAgam': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'GHcol': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'GHgam': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'GM': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'GNcol': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'GNgam': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'GQgam': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'GW': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'KE': <AlleleCountsDaskArray shape=(5506386, 4) dtype=int32>,
 'UGgam': <AlleleCo

In [14]:
allel.AlleleCountsChunkedTable(ac_subpops)

<AlleleCountsChunkedTable shape=(5506386,) dtype=[('AOcol', '<i4', (4,)), ('BFcol', '<i4', (4,)), ('BFgam', '<i4', (4,)), ('CIcol', '<i4', (4,)), ('CMgam', '<i4', (4,)), ('FRgam', '<i4', (4,)), ('GAgam', '<i4', (4,)), ('GHcol', '<i4', (4,)), ('GHgam', '<i4', (4,)), ('GM', '<i4', (4,)), ('GNcol', '<i4', (4,)), ('GNgam', '<i4', (4,)), ('GQgam', '<i4', (4,)), ('GW', '<i4', (4,)), ('KE', '<i4', (4,)), ('UGgam', '<i4', (4,))]
   nbytes=1.3G
   values=builtins.dict>